#### Выявляем корреляцию между длиной названия фильма и его оценкой

In [1]:
# импорт
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from funcs.spark_common import spark_read

In [2]:
# инициализация spark

spark = SparkSession \
    .builder \
    .appName("pyspark_imdb_corr") \
    .master("local[2]") \
    .config("spark.driver.memory", "10G") \
    .config("spark.jars", "postgresql-42.7.4.jar") \
    .getOrCreate()

In [3]:
# загружаем данные в датафреймы из таблиц БД
df_basics = spark_read(spark, "basics")
df_ratings = spark_read(spark, "ratings")

In [9]:
# фильтруем тип 'movie'
# join таблиц basics и ratings
# выбираем нужные колонки
# добавляем колонку с длиной названия фильма
df_join = df_basics \
    .where("titleType = 'movie'") \
    .join(df_ratings, df_basics.tconst == df_ratings.tconst) \
    .select(df_basics.primaryTitle, df_ratings.averageRating, df_ratings.numVotes) \
    .withColumn("primaryTitle_length", F.length(df_basics.primaryTitle))

# нормализация колонки numVotes, получение конечной метрики
min = df_join.agg({"numVotes": "min"}).collect()[0][0]
max = df_join.agg({"numVotes": "max"}).collect()[0][0]
df_join_norm = df_join. \
    withColumn('norm_numVotes', (F.col("numVotes") - min)/(max - min)). \
    withColumn("final_metric", (F.col("norm_numVotes") * F.col("averageRating"))). \
    drop("norm_numVotes")

# расчет корреляции
corr_norm = df_join_norm.select(F.round(F.corr("primaryTitle_length", "final_metric"), 3)).collect()[0][0]
corr_norm

-0.013